# day 22

https://adventofcode.com/2019/day/22

In [ ]:
import logging
import logging.config
import os

import yaml

In [ ]:
with open('../logging.yaml') as fp:
    logging_config = yaml.load(fp, Loader=yaml.FullLoader)

logging.config.dictConfig(logging_config)

In [ ]:
FNAME = os.path.join('data', 'day22.txt')

LOGGER = logging.getLogger('day22')

## part 1

### problem statement:

#### loading data

In [ ]:
test_0 = """deal with increment 7
deal into new stack
deal into new stack"""
answer_0 = [0, 3, 6, 9, 2, 5, 8, 1, 4, 7]

test_1 = """cut 6
deal with increment 7
deal into new stack"""
answer_1 = [3, 0, 7, 4, 1, 8, 5, 2, 9, 6]

test_2 = """deal with increment 7
deal with increment 9
cut -2"""
answer_2= [6, 3, 0, 7, 4, 1, 8, 5, 2, 9]

test_3 = """deal into new stack
cut -2
deal with increment 7
cut 8
cut -4
deal with increment 7
cut 3
deal with increment 9
deal with increment 3
cut -1"""
answer_3 = [9, 2, 5, 8, 1, 4, 7, 0, 3, 6]

In [ ]:
def load_data(fname=FNAME):
    with open(fname) as fp:
        return fp.read().strip()

#### function def

In [ ]:
import re

DEAL_WITH = 'deal with'
DEAL_INTO = 'deal into'
CUT = 'cut'

def parse_instr(i):
    if i[:9] == DEAL_WITH:
        instr_type = DEAL_WITH
        param = int(i.split(' ')[-1])
    elif i[:9] == 'deal into':
        instr_type = DEAL_INTO
        param = None
    elif i[:3] == CUT:
        instr_type = CUT
        param = int(i.split(' ')[-1])
    else:
        print(i)
        raise ValueError()
    return instr_type, param

In [ ]:
# [parse_instr(_) for _ in test_0.split('\n')]
# [parse_instr(_) for _ in test_1.split('\n')]
# [parse_instr(_) for _ in test_2.split('\n')]
# [parse_instr(_) for _ in test_3.split('\n')]
# [parse_instr(_) for _ in load_data().split('\n')]

In [ ]:
def apply_instr(l, instr_type, param=None):
    if instr_type == DEAL_WITH:
        L = len(l)
        new_list = {((i * param) % L): elem
                    for (i, elem) in enumerate(l)}
        return [new_list[i] for i in range(L)]
    elif instr_type == DEAL_INTO:
        l.reverse()
        return l
    elif instr_type == CUT:
        return l[param:] + l[:param]
        #if param > 0:
        #    for i in range(param):
        #        l.append(l.pop(0))
        #else:   
    else:
        msg = f"instr_type {instr_type} not known"
        LOGGER.error(msg)
        raise ValueError(msg)
    raise ValueError('should not have gotten here')

In [ ]:
# # deal into new deck
# l = list(range(10))
# l = apply_instr(l, DEAL_INTO)
# assert l == list(range(9, -1, -1))

# # cut 3
# l = list(range(10))
# l = apply_instr(l, CUT, 3)
# assert l == [3, 4, 5, 6, 7, 8, 9, 0, 1, 2]

# # cut -4
# l = list(range(10))
# l = apply_instr(l, CUT, -4)
# assert l == [6, 7, 8, 9, 0, 1, 2, 3, 4, 5,]

# # deal with increment 3
# l = list(range(10))
# l = apply_instr(l, DEAL_WITH, 3)
# assert l == [0, 7, 4, 1, 8, 5, 2, 9, 6, 3]

In [ ]:
def apply_all_instrs(l, instr_str):
    for i in instr_str.split('\n'):
        instr_type, param = parse_instr(i)
        l = apply_instr(l, instr_type, param)
    return l

In [ ]:
def q_1(data, L=10_007):
    l = list(range(L))
    return apply_all_instrs(l, data)

#### tests

In [ ]:
def test_q_1():
    LOGGER.setLevel(logging.DEBUG)
    tests = [(test_0, answer_0),
             (test_1, answer_1),
             (test_2, answer_2),
             (test_3, answer_3)]
    for (t, a) in tests:
        assert q_1(t, 10) == a
    LOGGER.setLevel(logging.INFO)

In [ ]:
test_q_1()

#### answer

In [ ]:
z = q_1(load_data())
z.index(2019)

## part 2

### problem statement:

#### function def

In [ ]:
import functools

In [ ]:
@functools.lru_cache()
def _deal_with(i, param, L):
    """i is some number which times param then mod L is
    the integer j, i.e. (i = (j * param) % L). this means
    there exists an n such that (j * param) = n * L + i.
    to invet this, we iterate through n until we find a
    value where (n * L + i) % param is 0
    
    """
    #n = 0
    #while True:
    #    if (n * L + i) % param == 0:
    #        return (n * L + i) / param
    # this is equivalent:
    accum = i
    while accum % param != 0:
        accum += L
    return accum / param


@functools.lru_cache()
def which_ends_up_at(i, instr_type, param=None, L=10):
    """which input list index (i_in) ends up in slot
    i in the output after instr_type
    
    """
    if instr_type == DEAL_WITH:
        return _deal_with(i, param, L)
    elif instr_type == DEAL_INTO:
        return L - i - 1
    elif instr_type == CUT:
        return (i + param) % L
    else:
        msg = f"instr_type {instr_type} not known"
        LOGGER.error(msg)
        raise ValueError(msg)
    raise ValueError('should not have gotten here')

In [ ]:
# deal into
assert which_ends_up_at(0, DEAL_INTO) == 9
assert which_ends_up_at(1, DEAL_INTO) == 8
assert which_ends_up_at(2, DEAL_INTO) == 7
assert which_ends_up_at(3, DEAL_INTO) == 6
assert which_ends_up_at(4, DEAL_INTO) == 5
assert which_ends_up_at(5, DEAL_INTO) == 4
assert which_ends_up_at(6, DEAL_INTO) == 3
assert which_ends_up_at(7, DEAL_INTO) == 2
assert which_ends_up_at(8, DEAL_INTO) == 1
assert which_ends_up_at(9, DEAL_INTO) == 0

# cut 3
assert which_ends_up_at(0, CUT, 3) == 3
assert which_ends_up_at(1, CUT, 3) == 4
assert which_ends_up_at(2, CUT, 3) == 5
assert which_ends_up_at(3, CUT, 3) == 6
assert which_ends_up_at(4, CUT, 3) == 7
assert which_ends_up_at(5, CUT, 3) == 8
assert which_ends_up_at(6, CUT, 3) == 9
assert which_ends_up_at(7, CUT, 3) == 0
assert which_ends_up_at(8, CUT, 3) == 1
assert which_ends_up_at(9, CUT, 3) == 2

# cut -4
assert which_ends_up_at(0, CUT, -4) == 6
assert which_ends_up_at(1, CUT, -4) == 7
assert which_ends_up_at(2, CUT, -4) == 8
assert which_ends_up_at(3, CUT, -4) == 9
assert which_ends_up_at(4, CUT, -4) == 0
assert which_ends_up_at(5, CUT, -4) == 1
assert which_ends_up_at(6, CUT, -4) == 2
assert which_ends_up_at(7, CUT, -4) == 3
assert which_ends_up_at(8, CUT, -4) == 4
assert which_ends_up_at(9, CUT, -4) == 5

# deal with increment 3
assert which_ends_up_at(0, DEAL_WITH, 3) == 0
assert which_ends_up_at(1, DEAL_WITH, 3) == 7
assert which_ends_up_at(2, DEAL_WITH, 3) == 4
assert which_ends_up_at(3, DEAL_WITH, 3) == 1
assert which_ends_up_at(4, DEAL_WITH, 3) == 8
assert which_ends_up_at(5, DEAL_WITH, 3) == 5
assert which_ends_up_at(6, DEAL_WITH, 3) == 2
assert which_ends_up_at(7, DEAL_WITH, 3) == 9
assert which_ends_up_at(8, DEAL_WITH, 3) == 6
assert which_ends_up_at(9, DEAL_WITH, 3) == 3

In [ ]:
import tqdm.autonotebook as tqdm

In [ ]:
def q_2(data):
    # basically, walk backwards through the instructions
    instructions = [parse_instr(_) for _ in data.split('\n')]
    L = 119_315_717_514_047
    
    j = 2020
    num_iter = 101_741_582_076_661
    
    i = 0
    while i < num_iter:
        if i % 100_000 == 0:
            LOGGER.info(f'i = {i:,}')
        # apply all instructions
        for (instr_type, param) in instructions[::-1]:
            j = which_ends_up_at(j, instr_type, param, L)
            
        if j == 2020:
            LOGGER.info('found cycle')
            LOGGER.debug(f'returned to input position after {i} steps')
            # skip to the last multiple of current i less than num_iter
            i = i * (num_iter // i)
        else:
            i += 1
    return i

#### tests

In [ ]:
# def test_q_2():
#     LOGGER.setLevel(logging.DEBUG)
#     assert q_2(test_data) == True
#     LOGGER.setLevel(logging.INFO)

In [ ]:
# test_q_2()

#### answer

In [ ]:
q_2(load_data())

fin

In [ ]:
import sys; sys.dont_write_bytecode = True

def do_case(inp: str, sample=False):
    # READ THE PROBLEM FROM TOP TO BOTTOM OK
    def sprint(*a, **k): sample and print(*a, **k)
    lines = inp.splitlines()
    cards = 119315717514047
    repeats = 101741582076661

    def inv(n):
        # gets the modular inverse of n
        # as cards is prime, use Euler's theorem
        return pow(n, cards-2, cards)
    def get(offset, increment, i):
        # gets the ith number in a given sequence
        return (offset + i * increment) % cards
    
    # increment = 1 = the difference between two adjacent numbers
    # doing the process will multiply increment by increment_mul.
    increment_mul = 1
    # offset = 0 = the first number in the sequence.
    # doing the process will increment this by offset_diff * (the increment before the process started).
    offset_diff = 0
    for line in inp.splitlines():
        if line == "deal into new stack":
            # reverse sequence.
            # instead of going up, go down.
            increment_mul *= -1
            increment_mul %= cards
            # then shift 1 left
            offset_diff += increment_mul
            offset_diff %= cards
        elif line.startswith("cut"):
            q = int(line.split(' ')[-1])
            # shift q left
            offset_diff += q * increment_mul
            offset_diff %= cards
        elif line.startswith("deal with increment "):
            q = int(line.split(' ')[-1])
            # difference between two adjacent numbers is multiplied by the
            # inverse of the increment.
            increment_mul *= inv(q)
            increment_mul %= cards

    def get_sequence(iterations):
        # calculate (increment, offset) for the number of iterations of the process
        # increment = increment_mul^iterations
        increment = pow(increment_mul, iterations, cards)
        # offset = 0 + offset_diff * (1 + increment_mul + increment_mul^2 + ... + increment_mul^iterations)
        # use geometric series.
        offset = offset_diff * (1 - increment) * inv((1 - increment_mul) % cards)
        offset %= cards
        return increment, offset

    increment, offset = get_sequence(repeats)
    print(get(offset, increment, 2020))
    
    return  # RETURNED VALUE DOESN'T DO ANYTHING, PRINT THINGS INSTEAD



do_case(load_data())